In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("train2.csv")

In [3]:
data = np.array(df)
m,n = data.shape
np.random.shuffle(data)
print(f"m:{m}")
print(f"n:{n}")

m:42000
n:785


In [4]:
df.head(5)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
val_data = data[0 : 2000].T
Y_val = val_data[0]
X_val = val_data[1 : n]
X_val = X_val / 255.

train_data = data[2000 : m].T
Y_train = train_data[0]
X_train = train_data[1 : n]
X_train = X_train / 255.

In [6]:
print(X_train.shape)
print(Y_train.shape)
print(X_val.shape)
print(Y_val.shape)

(784, 40000)
(40000,)
(784, 2000)
(2000,)


In [7]:
n_classes = len(set(Y_train))
print(n_classes)

10


In [8]:
from src.parameter_initialization import ParameterInitialization
from src.forward_propagation import ForwardPropagationForClassification
from src.back_propagation import BackPropagationForClassification
from src.cost_function import UpdateParameters
from src.evaluation import Evaluation

In [9]:
parameter_init = ParameterInitialization(X_train.shape[0], 2, 300, n_classes)
weights, biases = parameter_init.initialize_parameters()

In [10]:
forward_propagation = ForwardPropagationForClassification()
back_propagation = BackPropagationForClassification()
update_parameters = UpdateParameters()
evaluation = Evaluation(n_classes)

In [11]:
epochs = 100
learning_rate = 0.1

In [12]:
for i in range(epochs):
    forward_output = forward_propagation.forward_propagation_classification(weights, biases, X_train)
    back_output = back_propagation.back_propagation_classification(forward_output, weights, X_train, Y_train, m)
    weights, biases = update_parameters.update_parameters(weights, biases, back_output, learning_rate)
    key, value = list(forward_output.items())[-1]
    if i % 10 == 0:
        print("Epoch:", i)
        prediction = evaluation.predictions(value)
        print(f"Accuracy: {(evaluation.accuracy(prediction, Y_train))*100}")
        print("~~~~~~~~~~~~~~~~~") 

Epoch: 0
Accuracy: 9.925
~~~~~~~~~~~~~~~~~
Epoch: 10
Accuracy: 62.51
~~~~~~~~~~~~~~~~~
Epoch: 20
Accuracy: 72.56
~~~~~~~~~~~~~~~~~
Epoch: 30
Accuracy: 77.2775
~~~~~~~~~~~~~~~~~
Epoch: 40
Accuracy: 79.9025
~~~~~~~~~~~~~~~~~
Epoch: 50
Accuracy: 81.7375
~~~~~~~~~~~~~~~~~
Epoch: 60
Accuracy: 83.10499999999999
~~~~~~~~~~~~~~~~~
Epoch: 70
Accuracy: 84.135
~~~~~~~~~~~~~~~~~
Epoch: 80
Accuracy: 85.0625
~~~~~~~~~~~~~~~~~
Epoch: 90
Accuracy: 85.81
~~~~~~~~~~~~~~~~~


In [13]:
val_prediction = evaluation.validation_predictions(X_val, weights, biases)
print(f"The validation accuracy is {evaluation.accuracy(val_prediction, Y_val) * 100}")

TypeError: ForwardPropagationForClassification.forward_propagation_classification() takes 4 positional arguments but 5 were given